<a href="https://colab.research.google.com/github/ClaudeCoulombe/VIARENA/blob/master/Labos/Lab-Reconnaissance_Sonore/Distinction_Chants_Bruants.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Rappel - Fonctionnement d'un carnet web iPython

* Pour exécuter le code contenu dans une cellule d'un carnet iPython, cliquez dans la cellule et faites (⇧↵, shift-enter)
* Le code d'un carnet iPython s'exécute séquentiellement de haut en bas de la page. Souvent, l'importation d'une bibliothèque Python ou l'initialisation d'une variable est préalable à l'exécution d'une cellule située plus bas. Il est donc recommandé d'exécuter les cellules en séquence. Enfin, méfiez-vous des retours en arrière qui peuvent réinitialiser certaines variables.

<b>SVP</b>, déployez toutes les cellules en sélectionnant l'item « Développer les rubriques » de l'onglet « Affichage ».

# Distinction des chants de Bruants
## Labo avec des données sonores
    
    

## Inspiration et droits d'auteur

Ce laboratoire est basé sur le Tutoriel TensorFlow - <a href="https://www.tensorflow.org/tutorials/audio/simple_audio?hl=fr" target='_blank'>Reconnaissance audio simple / Reconnaître les mots-clés</a>.

Copyright (c) 2019-2024, The TensorFlow Authors.

Copyright (c) 2022-2024, Claude Coulombe

Le contenu de cette page est sous licence <a href="https://creativecommons.org/licenses/by/4.0/deed.fr" target='_blank'>Creative Commons Attribution 4.0 (CC BY 4.0)</a>,<br/>et les exemples de code sont sous <a href="https://www.apache.org/licenses/LICENSE-2.0" target='_blank'>licence Apache 2.0</a>.

### Données

Les données sur les chants d'oiseaux proviennent de de <a href="https://www.xeno-canto.org" target='_blank'>Xeno-canto</a>, un site Web de partage d'enregistrements de sons d'oiseaux sauvages du monde entier, une banque en données ouvertes sous licence <i>Creative Commons</i>.


## Importation bibliothèques Python

In [ ]:
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import tensorflow as tf
import keras
from keras import layers
from keras import models

from IPython import display

print("Bibliothèques Python chargées")

## Fixer le hasard pour la reproductibilité

La mise au point de réseaux de neurones implique certains processus aléatoires. Afin de pouvoir reproduire et comparer vos résultats d'expérience, vous fixez temporairement l'état aléatoire grâce à un germe aléatoire unique.

Pendant la mise au point, vous fixez temporairement l'état aléatoire pour la reproductibilité mais vous répétez l'expérience avec différents germes ou états aléatoires et prenez la moyenne des résultats.
<br/>
##### **Note**: Pour un système en production, vous ravivez simplement l'état  purement aléatoire avec l'instruction `GERME_ALEATOIRE = None`

In [ ]:
import os

# Définir un germe aléatoire
GERME_ALEATOIRE = 42

# Définir un état aléatoire pour Python
os.environ['PYTHONHASHSEED'] = str(GERME_ALEATOIRE)

# Définir un état aléatoire pour Python random
import random
random.seed(GERME_ALEATOIRE)

# Définir un état aléatoire pour NumPy
import numpy as np
np.random.seed(GERME_ALEATOIRE)

# Définir un état aléatoire pour TensorFlow
import tensorflow as tf
tf.random.set_seed(GERME_ALEATOIRE)
os.environ['TF_DETERMINISTIC_OPS'] = '1'
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

print("Germe aléatoire fixé")

## Acquisition des données

### Jeu de données - cris et chants d'oiseaux Xeno-canto

Ls données utilisées dans ce laboratoire proviennent de <a href="https://www.xeno-canto.org" target='_blank'>Xeno-canto</a> (XC). Xeno-canto est un site Web de partage d'enregistrements de sons d'oiseaux sauvages du monde entier. Il a été lancé en 2005 par Bob Planqué et Willem-Pier Vellinga. Xeno-canto est géré par la fondation Xeno-canto (ou officiellement Stichting Xeno-canto voor natuurgeluiden ) des Pays-Bas.

Notez que les données originalement en format .mp3 ont été converties en format .wav compatibles avec les outils TensorFlow.

Les données sonores originales sont publiées sur Xeno-canto sous licence <i>Creative Commons</i>. Grâce à l'IPA ultrasimple de Xeno-canto, l'auteur d'un enregistrement est facile à retrouver à partir du nom original du fichier d'enregistrement. Par exemple, avec XC<numéro>.wav, il suffit de taper https://xeno-canto.org/<numéro> dans un fureteur pour retrouver les détails de l'enregistrement.

Les <a href="https://www.kaggle.com/claudecoulombe/chants-de-bruants" target='_blank'>données sur le chant des bruants</a>  sont téléchargeables à partir du site Kaggle. Mais vous allez utiliser l'IPA (<i>API</i>) de Kaggle pour accélérer les transferts de données.

### Création des répertoires de données

Nous allons créer un répertoire de base `donnees`, un répertoire `lab_ecorces` où les données seront réparties en données d'entraînement, de validation et de test pour chaque classe cible.

Enfin, un répertoire `modeles` pour mémoriser les modèles une fois entraînés.

In [ ]:
!mkdir donnees
!mkdir "donnees/sons_bruants"
chemin_donnees = "donnees/sons_bruants"
repertoire_donnees = pathlib.Path(chemin_donnees)

### Utilisation de l'IPA (<i>API</i>) de Kaggle pour l'importation directe du jeu de données BarkNet

1. Commencez par installer la bibliothèque Python `kaggle`

In [ ]:
!pip3 install kaggle

2. Si ce n'est déjà fait, devenez membre de Kaggle avec votre adresse de courriel GMail:<br/>

<img src="https://courses.edx.org/asset-v1:UMontrealX+Cegep-Matane-VIARENA+2T2024+type@asset+block@Kaggle_API-1.png"/>

3. Maintenant, vous devez télécharger votre clé privée pour utiliser l'IPA de Kaggle.

4. Cliquez sur l'onglet « account » de votre profil Kaggle

<img src="https://courses.edx.org/asset-v1:UMontrealX+Cegep-Matane-VIARENA+2T2024+type@asset+block@Kaggle_API-2.png"/>

5. Sur la page « Account » cliquez sur le bouton « Create New API Token ».
    
<img style="margin-left:40px;" src="https://courses.edx.org/asset-v1:UMontrealX+Cegep-Matane-VIARENA+2T2024+type@asset+block@Kaggle_API-3.png"/>

6. Téléchargez votre clé privée « kaggle.json » pour l'IPA Kaggle dans un endroit temporaire sur votre poste de travail.

<img style="margin-left:40px;" src="https://courses.edx.org/asset-v1:UMontrealX+Cegep-Matane-VIARENA+2T2024+type@asset+block@Kaggle_API-5.png"/>

7. Maintenant, transférez (téléversez) votre clé privée « kaggle.json » dans votre environnement Colab. L'arborescence des fichiers dans Colab se trouve à gauche de la page du Notebook Colab.

La fenêtre de l'outil de fichiers de votre ordinateur s'ouvre alors. Allez chercher votre clé privée « kaggle.json » que vous avez sauvegardée sur votre  ordinateur.

<img style="margin-left:40px;" src="https://courses.edx.org/asset-v1:UMontrealX+Cegep-Matane-VIARENA+2T2024+type@asset+block@Colab_Importer_Fichier.png"/>

8. Créer à la racine un répertoire .kaggle et déplacez votre clé privée « kaggle.json » dans ce répertoire.

In [ ]:
!mkdir ~/.kaggle
!cp /content/kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle -all

9. Maintenant téléchargez le jeu de données « chants-de-bruants» avec la commande suivante:

In [ ]:
!kaggle datasets download -d claudecoulombe/chants-de-bruants/ --unzip -p "donnees/sons_bruants/"

Une fois les deux indicateurs ci-dessus à 100%, on réorganise les répertoires pour supprimer un niveau hiérarchique de répertoires inutile.

In [ ]:
# Réorganisation des répertoires

!mv donnees/sons_bruants/bruant_a_gorge_blanche/bruant_a_gorge_blanche/*.wav donnees/sons_bruants/bruant_a_gorge_blanche/
!rm -R donnees/sons_bruants/bruant_a_gorge_blanche/bruant_a_gorge_blanche/
!mv donnees/sons_bruants/bruant_chanteur/bruant_chanteur/*.wav donnees/sons_bruants/bruant_chanteur/
!rm -R donnees/sons_bruants/bruant_chanteur/bruant_chanteur/
!mv donnees/sons_bruants/bruant_des_pres/bruant_des_pres/*.wav donnees/sons_bruants/bruant_des_pres/
!rm -R donnees/sons_bruants/bruant_des_pres/bruant_des_pres/


### Répartition des données

In [ ]:
chants = np.array(tf.io.gfile.listdir(str(repertoire_donnees)))
chants = chants[chants != '.ipynb_checkpoints']
chants = chants[chants != '.DS_Store']
print('Chants:', chants)

#### Extraction et répartition aléatoire des clips audio dans une liste appelée "liste_de_fichiers_audio"

Le résultat est un structure de données de type matricielle appelée Tensor dans le jargon de TensorFlow

In [ ]:
liste_de_fichiers_audio = tf.io.gfile.glob(str(repertoire_donnees) + '/*/*')
liste_de_fichiers_audio = tf.random.shuffle(liste_de_fichiers_audio)
nombre_de_fichiers_audio = len(liste_de_fichiers_audio)
print('Nombre total de fichiers audio:', nombre_de_fichiers_audio)
print('Nombre de fichiers audio par classes:',
      len(tf.io.gfile.listdir(str(repertoire_donnees/chants[0]))))
print('Exemple de fichier audio:', liste_de_fichiers_audio[0])

#### Répartition des données d'entraînement, de validation et de tests

In [ ]:
# Ici la répartition est 80:10:10
fichiers_entrainement = liste_de_fichiers_audio[:144]
fichiers_validation = liste_de_fichiers_audio[144: 144 + 18]
fichiers_test = liste_de_fichiers_audio[-18:]

print("Taille du jeu de données d'entraînement:", len(fichiers_entrainement))
print('Taille du jeu de données de validation:', len(fichiers_validation))
print('Taille du jeu de données de test:', len(fichiers_test))

In [ ]:
print(fichiers_entrainement[:5])
print(fichiers_validation[:5])
print(fichiers_test[:5])

### Lecture des fichiers audio et des étiquettes de classes

Le prétraitement des données pour créer des représentations en ondes sonores binaires compatibles avec TensorFlow et les étiquettes de classes correspondantes.

Notez que:

* Chaque fichier .wav contient des données sonores formant une série chronologique ou série temporelle (<i>time series</i>) avec une fréquence d'échantillonnge fixe par seconde.
* Chaque échantillon représente l'amplitude du signal audio à un moment précis.
* Le taux d'échantillonnage pour cet ensemble de données en mp3 est généralement de 44.1 kHz ou 44 100 Hz
* La structure de données retournée par tf.audio.decode_wav est (échantillons, canaux), où canaux est 1 pour mono ou 2 pour stéréo. Le jeu de données contient surtout des enregistrements mono mais également des enegistrements stéréos.

In [ ]:
un_exemple_de_fichier_audio_wav = tf.io.read_file(chemin_donnees+"/bruant_chanteur/XC125704.wav")
test_audio, _ = tf.audio.decode_wav(contents=un_exemple_de_fichier_audio_wav)
test_audio.shape

## Prétraitement des données
#### Création de fonctions utilitaires, .wav vers TensorFlow
Définissons une fonction qui fait le prétraitement des fichiers audio .wav bruts de l'ensemble de données pour les transformer en des représentations audio compatibles avec TensorFlow:

In [ ]:
def decode_audio_wav(fichier_audio_wav):
    # Décode des fichiers binaires audio en format .wav vers un format TensorFlow 32 bits normalisé
    # dans l'intervalle [-1.0, 1.0]. Retourne une structure float32 audio et une fréquene d'échantillonnage
    # Les fichiers .wav ont deux canaux
    audio, _ = tf.audio.decode_wav(contents=fichier_audio_wav,desired_channels=1)
    # Puisque que toutes les données ont un seul canal (mono), supprimez l'axe "canal" du tableau.
    return tf.squeeze(audio, axis=-1)

In [ ]:
decode_audio_wav(un_exemple_de_fichier_audio_wav)

Maintenant, définissons une fonction qui crée des étiquettes en se basant sur les noms des répertoires parents pour chaque fichier. Nous allons utiliser `tf.RaggedTensors` car les structures matricielles (tenseurs) ont des dimensions irrégulières.



In [ ]:
def extraire_etiquette(chemin_fichier):
    morceaux_chemin = tf.strings.split(input=chemin_fichier,sep=os.path.sep)
    return morceaux_chemin[-2]

In [ ]:
extraire_etiquette(chemin_donnees+"/bruant_chanteur/XC125704.wav")

Enfin, définissons une fonction utilitaire `obtenir_onde_etiquette` qui rassemble les deux fonctions précédentes :

* L'entrée est le chemin du fichier audio .wav
* La sortie est un tuple contenant une représentation de l'onde sonore et son étiquette de classe. Donc une donnée prête pour un apprentissage supervisé.

In [ ]:
def obtenir_onde_etiquette(chemin_fichier_wav):
    etiquette = extraire_etiquette(chemin_fichier_wav)
    fichier_audio_wav = tf.io.read_file(chemin_fichier_wav)
    onde_sonore_tf = decode_audio_wav(fichier_audio_wav)
    return onde_sonore_tf, etiquette

In [ ]:
obtenir_onde_etiquette(chemin_donnees+"/bruant_chanteur/XC125704.wav")

#### Prétraitement des ensembles de données d'entraînement, de validation et de test

Nous alllons appliquer nos fonctions de prétraitement à l'ensemble d'entraînement, de validation et de test pour extraire les paires (représentation audio, étiquettes audio). En fait nous allons créer des générateurs de données de type `Dataset`.

Pour cela, nous allons utiliser `tf.data.Dataset` avec `Dataset.from_tensor_slices` et `Dataset.map`, en utilisant `obtenir_onde_etiquette` que nous avons défini précédemment.

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

donnees_entrainement = tf.data.Dataset.from_tensor_slices(fichiers_entrainement)

representations_ondes_sonores = donnees_entrainement.map(
    map_func=obtenir_onde_etiquette,
    num_parallel_calls=AUTOTUNE)

In [ ]:
fichiers_entrainement = tf.io.gfile.glob(str(chemin_donnees) + '/*/*')

print("5 premières données:\n",fichiers_entrainement[:5])
fichiers_entrainement = tf.random.shuffle(fichiers_entrainement)

print("\n5 premières données après mélange aléatoire:\n",fichiers_entrainement[:5])
donnees_entrainement = tf.data.Dataset.from_tensor_slices(fichiers_entrainement)

representations_ondes_sonores = donnees_entrainement.map(
    map_func=obtenir_onde_etiquette,
    num_parallel_calls=AUTOTUNE)

print("\nType representations_ondes_sonores:",type(representations_ondes_sonores))
print("Taille representations_ondes_sonores:",len(representations_ondes_sonores))

### Affichage d'échantillons de représentations sous la forme d'ondes sonores

In [ ]:
nombre_rangees = 3
nombre_colonnes = 3
nombre_donnees = nombre_rangees * nombre_colonnes
fig, axes = plt.subplots(nombre_rangees, nombre_colonnes, figsize=(1.62*14, 14))

for index_donnees, (onde_sonore, etiquette) in enumerate(representations_ondes_sonores.take(nombre_donnees)):
    rangee_courante = index_donnees // nombre_colonnes
    colonne_courante = index_donnees % nombre_colonnes
    ax = axes[rangee_courante][colonne_courante]
    ax.plot(onde_sonore.numpy())
    ax.set_yticks(np.arange(-1.2, 1.2, 0.2))
    etiquette = etiquette.numpy().decode('utf-8')
    ax.set_title(etiquette)

plt.show()

### Conversion des ondes en sonogrammes

Les représentations sous la forme d'ondes de notre jeu de données sont dans le domaine temporel. Elles montrent l'amplitude du signal en fonction du temps. Pour mieux les analyser sous forme d'images 2D avec des réseaux convolutifs, nous allons transformer ces ondes en spectrogrammes ou sonogrammes passant de signaux du domaine temporel en signaux du domaine temps-fréquence grâce à la <a href="https://fr.wikipedia.org/wiki/Transform%C3%A9e_de_Fourier_%C3%A0_court_terme" target='_blank'>transformée de Fourier locale</a> ou transformée de Fourier à fenêtre glissante, en anglais <i>Short Time Fourier Transform</i>, (<i>STFT</i>), pour convertir les formes d'onde en spectrogrammes, qui montrent les changements de fréquence au fil du temps et peuvent être représentés sous forme d'images 2D. Vous fournirez des images de sonogrammes en entrée à un réseau convolutif pour entraîner un modèle.

Une transformée de Fourier (`tf.signal.fft`) convertit un signal en ses fréquences composantes, mais perd toutes les informations temporelles. En comparaison, la transformée de Fourier à fenêtre glissante (`tf.signal.stft`) divise le signal en fenêtres de temps et exécute une transformée de Fourier sur chaque fenêtre, en préservant certaines informations temporelles et en renvoyant un tenseur 2D sur lequel on peut effectuer des convolutions.

#### Création d'une fonction utilitaire pour convertir les ondes sonores en sonogrammes

Les ondes doivent avoir la même longueur, de sorte que lorsqu'on les convertit en spectrogrammes, les résultats aient des dimensions similaires. Cela peut être fait en complétant avec des zéros les clips audio qui durent moins d'une seconde (en utilisant `tf.zeros`).

Lors de l'appel de `tf.signal.stft`, choisissez les paramètres `frame_length` et `frame_step` de sorte que "l'image" du spectrogramme généré soit presque carrée. Pour plus d'informations sur le choix des paramètres de `tf.signal.stft`, reportez-vous à cette <a href="https://www.coursera.org/lecture/audio-signal-processing/stft-2-tjEQe" target='_blank'>vidéo de Coursera en anglais</a> sur le traitement du signal audio.

La transformée de Fourier à fenêtre glissante produit un tableau de nombres complexes représentant l'amplitude et la phase. Cependant, dans ce laboratoire, vous n'utiliserez que l'amplitude, que vous pouvez dériver en appliquant `tf.abs` sur la sortie de `tf.signal.stft`.

In [ ]:
def generer_sonogramme(onde_sonore):
    # Compléter ou remplir avec des zéros si l'aonde sonore a moins de 41000 échantillon
    longueur_entree = 41000
    onde_sonore = onde_sonore[:longueur_entree]
    remplissage_de_zeros = tf.zeros([41000] - tf.shape(onde_sonore),dtype=tf.float32)
    # Convertir le type de l'onde sonore en un tenseur TensorFlow en float32
    onde_sonore = tf.cast(onde_sonore, dtype=tf.float32)
    # Concaténer l"onde sonore avec le `remplissage_de_zeros`, ce qui garantit que
     # tous les clips audio ont la même longueur.
    onde_sonore_longueur_normalisee = tf.concat([onde_sonore, remplissage_de_zeros], 0)
    # Convertir l'onde sonore en un sonogramme avec la fonction tf.signal.stft
    # i.e. avec une transformée de Fourier à fenêtre glissante
    sonogramme = tf.signal.stft(onde_sonore_longueur_normalisee, frame_length=255, frame_step=128)
    # Obtenir la magnitude
    sonogramme = tf.abs(sonogramme)
    # Ajouter une dimension `canaux`, afin que le sonogramme puisse être utilisé
    # en tant que données d'entrée de type image avec un réseau convolutif qui accepte
    # en entrée un matrice de dimensions (`taille_lot`, `hauteur`, `largeur`, `canaux`)..
    sonogramme = sonogramme[..., tf.newaxis]
    return sonogramme

### Affichage d'échantillons de représentations sous forme de sonogrammes

Explorons les données. Affichons l'onde sonore en représentation TensorFlow d'un exemple et son sonogramme correspondant, et écoutons le fichier audio d'origine :

In [ ]:
for onde_sonore_tf, etiquette in representations_ondes_sonores.take(2):
    etiquette = etiquette.numpy().decode('utf-8')
    sonogramme = generer_sonogramme(onde_sonore_tf)

print("Étiquette:", etiquette)
print("Dimensions de la représentation onde sonore:", onde_sonore_tf.shape)
print("Dimensions de la représentation sonogramme:", sonogramme.shape)
print('Écoute du fichier audio original')
display.display(display.Audio(onde_sonore_tf, rate=41000))

Maintenant, définissons une fonction pour afficher un sonogramme :

In [ ]:
def afficher_sonogramme(sonogramme, ax):
    if len(sonogramme.shape) > 2:
        assert len(sonogramme.shape) == 3
        sonogramme = np.squeeze(sonogramme, axis=-1)
    # Convert the frequencies to log scale and transpose, so that the time is
    # represented on the x-axis (columns).
    # Add an epsilon to avoid taking a log of zero.
    # Convertissez les fréquences sur une échelle logarithmique et transposez,
    # de sorte que le temps soit représenté sur l'axe des abscisses (colonnes).
     # Ajoutez un epsilon pour éviter de calculer le logarithmique de zéro.
    log_sonogramme = np.log(sonogramme.T + np.finfo(float).eps)
    hauteur = log_sonogramme.shape[0]
    largeur = log_sonogramme.shape[1]
    X = np.linspace(0, np.size(sonogramme), num=largeur, dtype=int)
    Y = range(hauteur)
    ax.pcolormesh(X, Y, log_sonogramme, shading='auto')

Affichons la représentation de l'onde sonore dans le temps et le spectrogramme correspondant (fréquences dans le temps) :

In [ ]:
fig, axes = plt.subplots(2, figsize=(1.62*8, 8))
timescale = np.arange(onde_sonore_tf.shape[0])
axes[0].plot(timescale, onde_sonore_tf.numpy())
axes[0].set_title('Onde sonore')
axes[0].set_xlim([0, 41000])

afficher_sonogramme(sonogramme.numpy(), axes[1])
axes[1].set_title('Sonogramme')
plt.show()

Maintenant, définissons une fonction qui transforme les jeux de données contenant des ondes sonores en sonogrammes et leurs étiquettes correspondantes en ID entiers :

In [ ]:
def obtenir_sonogramme_etiquetteID(onde_sonore, etiquette):
    sonogramme = generer_sonogramme(onde_sonore)
    etiquette_id = tf.argmax(etiquette == chants)
    return sonogramme, etiquette_id

Appliquons `get_spectrogram_and_label_id` sur nos jeux de données avec la fonction `Dataset.map`:



In [ ]:
representations_sonogrammmes = representations_ondes_sonores.map(
  map_func=obtenir_sonogramme_etiquetteID,
  num_parallel_calls=AUTOTUNE)

### Affichage d'échantillons de représentations sous la forme de sonogrammes

In [ ]:
nombre_rangees = 3
nombre_colonnes = 3
nombre_donnees = nombre_rangees * nombre_colonnes
fig, axes = plt.subplots(nombre_rangees, nombre_colonnes, figsize=(1.62*12, 12))

for index_donnees, (sonogramme, etiquette) in enumerate(representations_sonogrammmes.take(nombre_donnees)):
    rangee_courante = index_donnees // nombre_colonnes
    colonne_courante = index_donnees % nombre_colonnes
    ax = axes[rangee_courante][colonne_courante]
    afficher_sonogramme(sonogramme.numpy(), ax)
    ax.set_title(chants[etiquette.numpy()])
    ax.axis('off')

plt.show()

### Appliquer le prétraitement sur les jeux de données de validation et de test :

In [ ]:
def pretraiter_jeux_donnees(chemin_fichier):
    fichier_donnees = tf.data.Dataset.from_tensor_slices(chemin_fichier)
    representations_sortie = fichier_donnees.map(
        map_func=obtenir_onde_etiquette,
        num_parallel_calls=AUTOTUNE)
    representations_sortie = representations_sortie.map(
        map_func=obtenir_sonogramme_etiquetteID,
        num_parallel_calls=AUTOTUNE)
    return representations_sortie

In [ ]:
sonogrammes_entrainement = representations_sonogrammmes
sonogrammes_validation = pretraiter_jeux_donnees(fichiers_validation)
sonogrammes_test = pretraiter_jeux_donnees(fichiers_test)

## Construction et entraînement d'un modèle

### Découpage des données en lots pour l'entraînement

In [ ]:
taille_lot = 64
sonogrammes_entrainement = sonogrammes_entrainement.batch(taille_lot)
sonogrammes_validation = sonogrammes_validation.batch(taille_lot)

### Ajout des opérations de cache `Dataset.cache` et de pré-lecture `Dataset.prefetch`

Afin de réduire la latence de lecture lors de l'entraînement du modèle

In [ ]:
sonogrammes_entrainement = sonogrammes_entrainement.cache().prefetch(AUTOTUNE)
sonogrammes_validation = sonogrammes_validation.cache().prefetch(AUTOTUNE)

### Construction du modèle

Pour le modèle, nous utiliserons un réseau convolutif, puisque nous avons transformé les fichiers audio en images de sonogrammes.

Notre modèle `tf.keras.Sequential` utilisera les couches de prétraitement suivantes :

* `tf.keras.layers.Resizing` : pour sous-échantillonner l'entrée afin de permettre au modèle de s'entraîner plus rapidement.
* `tf.keras.layers.Normalization` : pour normaliser chaque pixel de l'image en fonction de sa moyenne et de son écart-type.

Pour la couche de normalisation, la méthode `adapt` devrait être appelée sur les données d'entraînement afin de calculer des statistiques agrégées (c'est-à-dire la moyenne et l'écart type).

In [ ]:
for sonogramme, _ in representations_sonogrammmes.take(1):
    dimensions_entree = sonogramme.shape
print('Dimensions Entree:', dimensions_entree)
nombre_classes = len(chants)

# Créer une couche de normalisation Keras
couche_normalisation = layers.Normalization()
# Ajuster la couche de normalisation au sonogramme avec la fonction `adapt`
couche_normalisation.adapt(data=representations_sonogrammmes.map(map_func=lambda spec, label: spec))

modele = models.Sequential([
    layers.Input(shape=dimensions_entree),
    # Sous-échantillonner l'entrée
    layers.Resizing(32, 32),
    # Normalisation
    couche_normalisation,
    layers.Conv2D(32, 3, activation='relu'),
    layers.Conv2D(64, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.25),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(nombre_classes),
])

#
modele.summary()

### Compilation du modèle

Compilons le modèle Keras avec l'optimiseur `Adam` et la perte d'entropie croisée :

In [ ]:
modele.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy'],
)

### Entraînement du modèle


In [ ]:
iterations = 20
traces_entrainement = modele.fit(
    sonogrammes_entrainement,
    validation_data=sonogrammes_validation,
    epochs=iterations,
    #callbacks=tf.keras.callbacks.EarlyStopping(verbose=1, patience=3),
)

### Affichage des courbes d'entraînement

In [ ]:
plt.figure(figsize=(1.62*6, 6))
metriques = traces_entrainement.history
plt.plot(traces_entrainement.epoch, metriques['loss'], metriques['val_loss'])
plt.legend(["erreur d'entraînement ", "erreur de validation"])
plt.xlabel("Nombre d'itérations")
plt.ylabel("Erreur d'entropie")
plt.show()

In [ ]:
plt.figure(figsize=(1.62*6, 6))
metriques = traces_entrainement.history
plt.plot(traces_entrainement.epoch, metriques['accuracy'], metriques['val_accuracy'])
plt.legend(["exactitude d'entraînement", 'exactitude de validation'])
plt.xlabel("Nombre d'itérations")
plt.ylabel("Exactitude")
plt.show()

## Évaluation du modèle

Nous allons évaluer la performance du modèle sur les données de test

In [ ]:
test_audio = []
test_etiquettes = []

for audio, etiquette in sonogrammes_test:
    test_audio.append(audio.numpy())
    test_etiquettes.append(etiquette.numpy())

test_audio = np.array(test_audio)
test_labels = np.array(test_etiquettes)

In [ ]:
classes_predites = np.argmax(modele.predict(test_audio), axis=1)
vraies_classes = test_etiquettes

exactitude_test = sum(classes_predites == vraies_classes) / len(vraies_classes)
print(f'Exactitude sur les données de test: {exactitude_test:.0%}')

### Affichage d'une matrice de confusion

In [ ]:
confusion_mtx = tf.math.confusion_matrix(vraies_classes, classes_predites)
plt.figure(figsize=(10, 8))
sns.heatmap(confusion_mtx,
            xticklabels=chants,
            yticklabels=chants,
            annot=True, fmt='g')
plt.xlabel('Classes prédites')
plt.ylabel('Vraies classes')
plt.show()

## Test en inférence sur des données fraîches

Enfin, vérifiez la prédiction du modèle à l'aide d'un nouveau fichier audio pris sur Xeno-canto. Quelle est la performance de votre modèle ?

In [ ]:
!wget "https://xeno-canto.org/550411/download" -O "550411-bruant_des_pres.mp3"
chemin_fichier_mp3 = "550411-bruant_des_pres.mp3"

### Conversion de .mp3 à .wav

In [ ]:
# Installation bibliothèque de conversion .mp3 à .wav
!pip3 install pydub

In [ ]:
import os
from os import path
from pydub import AudioSegment

fichier_mp3 = AudioSegment.from_mp3(chemin_fichier_mp3)
chemin_fichier_wav = "donnees/sons_bruants/bruant_des_pres/Fichier_TEST"
fichier_mp3.export(chemin_fichier_wav,format="wav")

In [ ]:
sonogrammes_inference = pretraiter_jeux_donnees([str(chemin_fichier_wav)])

for sonogramme, etiquettte in sonogrammes_inference.batch(1):
    classe_predite = modele(sonogramme)
    plt.bar(chants, tf.nn.softmax(classe_predite[0]))
    plt.title(f'Prediction: "{chants[etiquettte[0]]}"')
    plt.show()

In [ ]:
print("Exécution du carnet web IPython terminée")